# 3. Add sidewalk widths to basic pedestrian network

In [ ]:
# Standard library and path imports
import set_path
import math
import warnings

# Third-party library imports
import numpy as np
import pandas as pd
import shapely.ops as so
import shapely.geometry as sg
import geopandas as gpd
from geopandas import GeoDataFrame
import networkx as nx
import momepy
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
import folium

# Local or project-specific imports
import plot_utils
import poly_utils
import settings as st

if st.my_run == "azure":
    import config_azure as cf
elif st.my_run == "local":
    import config as cf

from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

## Import basic network data

In [ ]:
# Get basic pedestrian network and sidewalks
df_bgt_nw = gpd.read_file(cf.output_basic_network)
df_bgt_sw = gpd.read_file(cf.output_sidewalks_basic_network)

In [ ]:
df_bgt_nw.rename(columns={'geometry': 'centerlines'}, inplace=True)
df_bgt_exp = pd.merge(df_bgt_nw, df_bgt_sw, on='sidewalk_id', how='left')
df_bgt_exp = df_bgt_exp.set_geometry('centerlines')
df_bgt_exp.head(3)

## Create relevant variable - full width

### Get width and color

In [ ]:
df_bgt_exp[['avg_width', 'min_width']] = df_bgt_exp.progress_apply(
    lambda row: poly_utils.get_avg_width_cl(row.geometry, row.centerlines, 
                                            st.width_resolution, st.width_precision), axis=1)

In [ ]:
conditions = [
    (df_bgt_exp['min_width'] < st.width_1),
    (df_bgt_exp['min_width'] >= st.width_1) & (df_bgt_exp['min_width'] < st.width_2),
    (df_bgt_exp['min_width'] >= st.width_2) & (df_bgt_exp['min_width'] < st.width_3),
    (df_bgt_exp['min_width'] >= st.width_3) & (df_bgt_exp['min_width'] < st.width_4),
    (df_bgt_exp['min_width'] >= st.width_4) & (df_bgt_exp['min_width'] < st.width_5),
    (df_bgt_exp['min_width'] >= st.width_5) & (df_bgt_exp['min_width'] < st.width_6),
    (df_bgt_exp['min_width'] >= st.width_6)
]

values_float = [st.min_path_width, st.width_1, st.width_2, st.width_3, st.width_4, st.width_5, st.width_6]
values_indication = ['<' + str(st.width_1) + 'm', 
                     str(st.width_1) + '-' + str(st.width_2) + 'm', 
                     str(st.width_2) + '-' + str(st.width_3) + 'm', 
                     str(st.width_3) + '-' + str(st.width_5) + 'm', 
                     str(st.width_4) + '-' + str(st.width_5) + 'm', 
                     str(st.width_5) + '-' + str(st.width_6) + 'm', 
                     '>' + str(st.width_6) + 'm']

In [ ]:
df_bgt_exp['full_width_float'] = np.select(conditions, values_float)
df_bgt_exp['full_width'] = np.select(conditions, values_indication)
df_bgt_exp['full_width'].value_counts()

### Remove too narrow paths and short-ends

In [ ]:
# Apply minimal path width on BGT centerlines
print(df_bgt_exp.shape)
df_bgt_exp = df_bgt_exp[df_bgt_exp['min_width'] > st.min_path_width].reset_index(drop=True)
print(df_bgt_exp.shape)

In [ ]:
# Remove short lines
df_bgt_exp = df_bgt_exp.rename(columns={'geometry':'geometry_sidewalks', 'centerlines':'geometry'}) 
mls_per_id = poly_utils.create_mls_per_sidewalk(df_bgt_exp, crs=st.CRS)
mls_per_id['geometry'] = mls_per_id['geometry'].progress_apply(
    lambda x: poly_utils.remove_short_lines(x, st.min_se_length_fw))

In [ ]:
# Apply selection of longer lines to original dataframe
long_segments_df = gpd.GeoDataFrame(mls_per_id.geometry.explode())
df_bgt_exp = df_bgt_exp.merge(long_segments_df, how='inner')
df_bgt_exp.shape

## Import segments data

In [ ]:
# Read lines with widths (calculated using notebook 5 of https://github.com/Amsterdam-AI-Team/Urban_PointCloud_Sidewalk_Width)
df_segments_full = gpd.read_file(cf.segments_file, crs=st.CRS)

### Selection of segments within pilot area

In [ ]:
# Import areas
df_areas = gpd.read_file(cf.output_pilot_area)

# Only keep BGT data within pilot areas
df_segments = df_segments_full.sjoin(df_areas, how='inner', predicate='within')  # note: only sidewalk polygons fully inside area are included

## Create relevant variable - obstacle-free width

### Remove too narrow paths and short-ends

In [ ]:
# Apply minimal path width
df_segments_wide = df_segments[df_segments['min_width'] > st.min_path_width].reset_index(drop=True)
print(df_segments.shape)
print(df_segments_wide.shape)

In [ ]:
# Remove short lines (this takes a while)
mls_per_id = poly_utils.create_mls_per_sidewalk(df_segments_wide, crs=st.CRS)
mls_per_id['geometry'] = mls_per_id['geometry'].progress_apply(poly_utils.remove_short_lines)

In [ ]:
# Apply selection of longer lines to original dataframe
long_segments_df = gpd.GeoDataFrame(mls_per_id.geometry.explode())
df_segments_wide = df_segments_wide.merge(long_segments_df, how='inner')
df_segments_wide.shape

### Apply width factor

In [ ]:
# Add width factor, for calculating the weights of the paths later
conditions = [
    (df_segments_wide['min_width'] < st.width_1),
    (df_segments_wide['min_width'] >= st.width_1) & (df_segments_wide['min_width'] < st.width_2),
    (df_segments_wide['min_width'] >= st.width_2) & (df_segments_wide['min_width'] < st.width_3),
    (df_segments_wide['min_width'] >= st.width_3) & (df_segments_wide['min_width'] < st.width_4),
    (df_segments_wide['min_width'] >= st.width_4) & (df_segments_wide['min_width'] < st.width_5),
    (df_segments_wide['min_width'] >= st.width_5) & (df_segments_wide['min_width'] < st.width_6),
    (df_segments_wide['min_width'] >= st.width_6)
]
values_factor = [1000000000000, 10000000000, 100000000, 1000000, 10000, 100, 1]
df_segments_wide['min_width_factor'] = np.select(conditions, values_factor).astype('int64')

### Take point cloud coverage into account

In [ ]:
df_segments_wide['min_width_factor'][df_segments_wide['pc_coverage'] == False] = 100000000000001

In [ ]:
df_segments_wide['min_width_factor'].value_counts()

### Do network calculation

In [ ]:
df_bgt_exp['route_weight'] = np.nan
print('start doing network calculation on ' + str(len(df_bgt_sw['sidewalk_id'])) + ' rows')

In [ ]:
# Create final dataframe
final_df = pd.DataFrame()

for i in range(len(df_bgt_sw['sidewalk_id'])):
    print(i)
      
    # Get sidewalk polygon for this centerline 
    my_sidewalk = df_bgt_sw['geometry'].values[i]   
    
    # Create graph for all paths withing this sidewalk polygon
    df_sidewalk = df_segments_wide[df_segments_wide['geometry'].within(my_sidewalk)].reset_index(drop=True)
    G = momepy.gdf_to_nx(df_sidewalk, approach="primal", multigraph=True)
    
    # Create dataframe with linestrings of centerline
    centerline_df = df_bgt_exp[df_bgt_exp['sidewalk_id'] == i].reset_index(drop=True)
    
    for j in range(len(centerline_df['geometry'])):  
        
        # Get line
        my_line = centerline_df.iloc[[j]]['geometry'].values[0]
        
        if len(list(G.nodes)) > 0:
            # Check if my_line has start and end (not a ring)
            if len(my_line.boundary) == 0:
                print('no route calculated for line (j)', j, 'in sidewalk (i)', i, '(ring)')
                centerline_df['route_weight'][j] = 0
            else:
                # Get origin and destination location
                origin_point, dest_point = my_line.boundary
                origin_node_loc = so.nearest_points(origin_point, sg.MultiPoint(list(G.nodes)))[1]
                dest_node_loc = so.nearest_points(dest_point, sg.MultiPoint(list(G.nodes)))[1]

                # Get origin and destination node
                origin_node = (origin_node_loc.x, origin_node_loc.y)
                dest_node = (dest_node_loc.x, dest_node_loc.y)   

                # Get weight of optimal route in graph
                try:
                    route_weight = nx.shortest_path_length(G, origin_node, dest_node,
                                                           weight='min_width_factor')
                    if (origin_point.distance(origin_node_loc) < st.max_dist) and (dest_point.distance(dest_node_loc) < st.max_dist):
                        centerline_df['route_weight'][j] = route_weight
                    else:
                        print('origin and/or destination node too far from line start/end for line (j)', j, 'in sidewalk (i)', i) 
                        centerline_df['route_weight'][j] = np.nan
                except nx.NetworkXNoPath:
                    print('no route found for line (j)', j, 'in sidewalk (i)', i)
                    centerline_df['route_weight'][j] = 100000000000000
        else:
            print('network has zero nodes')

    # Append data to final dataframe
    final_df = pd.concat([final_df, centerline_df])
final_df = final_df.reset_index()

In [ ]:
# Get final width of the routes
final_df['obstacle_free_width_float'] = final_df.progress_apply(    
    lambda row: poly_utils.get_route_width(row.route_weight), axis=1)

## Post-process 

In [ ]:
final_df = final_df.set_geometry('geometry')

In [ ]:
final_df['obstacle_free_width_float'].value_counts(dropna=False)

### Fix incorrect labeling of obstacle-free width (> full width)

In [ ]:
# Start a column to track any adjustments we make to the obstacle-free width
final_df['width_fill'] = np.where(final_df['obstacle_free_width_float'] > final_df['full_width_float'], 1, 0)

# If obstacle free width float is bigger than full width, set obstacle free width float to full width
final_df['obstacle_free_width_float'] = np.where(final_df['obstacle_free_width_float'] > final_df['full_width_float'], final_df['full_width_float'], final_df['obstacle_free_width_float'])

### Fix unknown widths

In [ ]:
# Fill unknown widths with minimum width of neighbors
for i, row in final_df.iterrows():
    if math.isnan(row['obstacle_free_width_float']):
        my_edge = row['geometry']
        my_dist = final_df['geometry'].distance(my_edge)
        my_nb_ids = my_dist.loc[my_dist == 0].index.tolist()
        new_width = min(final_df['obstacle_free_width_float'].iloc[my_nb_ids])
        if math.isnan(new_width):
            # If no neighbors with known width are found, fill with lowest width category
            final_df.loc[i, 'obstacle_free_width_float'] = st.min_path_width
            final_df.loc[i, 'width_fill'] = 3
        else:
            final_df.loc[i, 'obstacle_free_width_float'] = new_width
            final_df.loc[i, 'width_fill'] = 2


In [ ]:
final_df['obstacle_free_width_float'].value_counts(dropna=False)

In [ ]:
final_df['width_fill'].value_counts(dropna=False)

### Set final indication

In [ ]:
# Get dataframe with only valid lines
final_df_select = final_df[final_df['obstacle_free_width_float'] != 0]
final_df_select = final_df_select.reset_index(drop=True)

In [ ]:
# Add meter indication
conditions = [
    (final_df_select['obstacle_free_width_float'] == st.min_path_width),
    (final_df_select['obstacle_free_width_float'] == st.width_1),   
    (final_df_select['obstacle_free_width_float'] == st.width_2),
    (final_df_select['obstacle_free_width_float'] == st.width_3),
    (final_df_select['obstacle_free_width_float'] == st.width_4),
    (final_df_select['obstacle_free_width_float'] == st.width_5),  
    (final_df_select['obstacle_free_width_float'] == st.width_6),  
    (final_df_select['obstacle_free_width_float'] == np.nan)
]
values_indication = ['<' + str(st.width_1) + 'm', 
                     str(st.width_1) + '-' + str(st.width_2) + 'm', 
                     str(st.width_2) + '-' + str(st.width_3) + 'm', 
                     str(st.width_3) + '-' + str(st.width_4) + 'm', 
                     str(st.width_4) + '-' + str(st.width_5) + 'm', 
                     str(st.width_5) + '-' + str(st.width_6) + 'm', 
                     '>' + str(st.width_6) + 'm',
                    'unknown']
final_df_select['obstacle_free_width'] = np.select(conditions, values_indication)

### Adjust crs

In [ ]:
final_df_select = final_df_select.set_crs(st.CRS)  # TODO might be removed?
print('finished post-processing')

## Merge with basic network

In [ ]:
# Select relevant columns for merging
final_df_select = final_df_select[['geometry', 'obstacle_free_width', 'obstacle_free_width_float', 'width_fill', 
                                   'full_width', 'full_width_float']]

In [ ]:
# Do inner join with original network
df_combined = pd.merge(df_bgt_nw, final_df_select, how='inner', left_on='centerlines', right_on='geometry')

## Store

In [ ]:
df_store = df_combined[['cl_id', 'centerlines', 'length', 'sidewalk_id', 'obstacle_free_width', 'obstacle_free_width_float', 'width_fill']]
df_store = df_store.set_geometry('centerlines').set_crs(st.CRS)
df_store.head()

In [ ]:
df_store.to_file(cf.output_file_widths, driver='GPKG') 
print('stored final data')

## Visualize network including widths

In [ ]:
# set True for satelite background, False for standard background
satellite = False

# Create Folium map
map = folium.Map(
    location=[52.350547922223434, 4.794019242371844], tiles=plot_utils.generate_map_params(satellite=satellite),
    min_zoom=10, max_zoom=25, zoom_start=17,
    zoom_control=True, control_scale=True, control=False
    )

# Add network with widths
folium.GeoJson(data=df_store[df_store['obstacle_free_width_float'] == st.min_path_width], style_function=lambda x: {"color": "darkred"}).add_to(map)
folium.GeoJson(data=df_store[df_store['obstacle_free_width_float'] == st.width_1], style_function=lambda x: {"color": "red"}).add_to(map)
folium.GeoJson(data=df_store[df_store['obstacle_free_width_float'] == st.width_2], style_function=lambda x: {"color": "orange"}).add_to(map)
folium.GeoJson(data=df_store[df_store['obstacle_free_width_float'] == st.width_3], style_function=lambda x: {"color": "yellow"}).add_to(map)
folium.GeoJson(data=df_store[df_store['obstacle_free_width_float'] == st.width_4], style_function=lambda x: {"color": "greenyellow"}).add_to(map)
folium.GeoJson(data=df_store[df_store['obstacle_free_width_float'] == st.width_5], style_function=lambda x: {"color": "limegreen"}).add_to(map)
folium.GeoJson(data=df_store[df_store['obstacle_free_width_float'] == st.width_6], style_function=lambda x: {"color": "green"}).add_to(map)

map 

In [ ]:
# Store map
map.save(cf.network_map_widths)